**Estudiante:** Juan Pablo Nieto Cortes  
**Asignatura:** AREP Arquitectura Empresarial

# Regresión Logística desde Cero para Predicción de Enfermedad Cardíaca

## 1. Introducción y Contexto

En este proyecto, implementaremos un modelo de **Regresión Logística** desde cero, utilizando únicamente librerías fundamentales como **NumPy**, **Pandas** y **Matplotlib**. El objetivo es predecir la presencia de enfermedad cardíaca en pacientes basándonos en un conjunto de atributos clínicos.

### ¿Qué es la Regresión Logística?
La regresión logística es un algoritmo de aprendizaje supervisado utilizado para problemas de **clasificación binaria**. A diferencia de la regresión lineal, que predice valores continuos, la regresión logística estima la probabilidad de que una instancia pertenezca a una clase particular (por ejemplo, "Enfermo" vs "Sano") utilizando la función sigmoide para mapear las salidas a un rango entre 0 y 1.

### Importancia en Medicina
En el contexto médico, la capacidad de predecir el riesgo de enfermedad cardíaca (una de las principales causas de muerte a nivel mundial) a partir de datos no invasivos o mínimamente invasivos es crucial. Un modelo bien calibrado puede servir como herramienta de apoyo para el diagnóstico temprano.

### Dataset
Utilizaremos el dataset de **Predicción de Enfermedad Cardíaca** (disponible en Kaggle/neurocipher), que contiene 270 muestras con 13 atributos clínicos (como edad, presión arterial, colesterol, etc.) y la variable objetivo que indica la presencia o ausencia de enfermedad cardíaca.

### Objetivo del Notebook
El propósito principal es educativo: entender los fundamentos matemáticos detrás del entrenamiento de modelos. Implementaremos manualmente:
- La función de hipótesis (Sigmoide).
- La función de costo (Log Loss).
- El cálculo de gradientes.
- El algoritmo de optimización Gradient Descent.
- Regularización L2 para evitar overfitting.


## 2. Carga y Exploración del Dataset (EDA)

Comenzaremos cargando los datos y realizando un análisis exploratorio básico para entender la estructura del dataset, los tipos de datos y la distribución de las clases.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Configuración de visualización
plt.style.use('ggplot')

# Ruta al dataset (ajustada a la ubicación de descarga)
dataset_path = "Heart_Disease_Prediction.csv"

# Cargar dataset
df = pd.read_csv(dataset_path)

# Mostrar primeras filas
print("Primeras filas del dataset:")
display(df.head())

# Información general
print("\nInformación del dataset:")
print(df.info())

# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
display(df.describe())

# Verificar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())


FileNotFoundError: [Errno 2] No such file or directory: '\\Logistic-Regression\\Heart_Disease_Prediction.csv'

### Análisis de la Variable Objetivo y Distribuciones

A continuación, visualizaremos la distribución de la variable objetivo `Heart Disease` para ver si las clases están balanceadas. También observaremos histogramas de algunas variables numéricas clave para entender sus rangos y formas.

Es necesario convertir la variable objetivo a formato binario (0 y 1) para la regresión logística. Asumiremos "Presence" como 1 y "Absence" como 0.


In [4]:
# Convertir variable objetivo a binaria
# Verificamos los valores únicos primero
print("Valores únicos en 'Heart Disease':", df['Heart Disease'].unique())

df['Target'] = df['Heart Disease'].apply(lambda x: 1 if x == 'Presence' else 0)

# 1. Distribución de clases
plt.figure(figsize=(6, 4))
df['Target'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Distribución de Clases (0: Ausencia, 1: Presencia)')
plt.xlabel('Clase')
plt.ylabel('Frecuencia')
plt.xticks(rotation=0)
plt.show()

print("Conteo de clases:")
print(df['Target'].value_counts())

# 2. Histogramas de variables clave
features_to_plot = ['Age', 'BP', 'Cholesterol', 'Max HR']
df[features_to_plot].hist(figsize=(10, 8), bins=20, edgecolor='black')
plt.suptitle('Histogramas de Variables Numéricas Clave')
plt.show()


NameError: name 'df' is not defined

### Observaciones del EDA
- **Balance de clases**: Observamos las barras para determinar si existe un desbalance significativo. Si las clases están relativamente equilibradas, no necesitaremos técnicas complejas de re-muestreo.
- **Variables Numéricas**:
    - `Age`: Distribución de edades de los pacientes. Es un factor de riesgo importante.
    - `BP` (Presión Arterial): Valores altos indican hipertensión.
    - `Cholesterol`: Niveles altos son factor de riesgo.
    - `Max HR`: Frecuencia cardíaca máxima alcanzada.
- **Ausencia de nulos**: `df.info()` nos confirmó si hay datos faltantes que requieran imputación.


## 3. Preprocesamiento de Datos

La regresión logística entrenada con Gradient Descent es sensible a la escala de las características. Si una variable tiene rangos de 0-1000 (como colesterol) y otra de 0-1 (como sexo), el gradiente oscilará mucho y tardará en converger.

Pasos a realizar:
1.  **Selección de Features**: Elegiremos al menos 6 variables predictoras relevantes.
2.  **Normalización (Z-score)**: Restar la media y dividir por la desviación estándar ($\frac{x - \mu}{\sigma}$).
3.  **Split Train/Test Estratificado**: Dividir los datos manteniendo la proporción de clases en ambos conjuntos (70% entrenamiento, 30% prueba).


In [ ]:
# 1. Selección de Features (al menos 6)
# Usaremos: Age, Sex, BP, Cholesterol, Max HR, ST depression
selected_features = ['Age', 'Sex', 'BP', 'Cholesterol', 'Max HR', 'ST depression']
X_raw = df[selected_features].values
y = df['Target'].values

print(f"Features seleccionadas: {selected_features}")
print(f"Dimensiones X: {X_raw.shape}, y: {y.shape}")

# 2. Separación Train/Test Estratificado Manual
# Indices de cada clase
idx_0 = np.where(y == 0)[0]
idx_1 = np.where(y == 1)[0]

# Shuffle indices
np.random.seed(42)
np.random.shuffle(idx_0)
np.random.shuffle(idx_1)

# Tamaño de train (70%)
split_ratio = 0.7
n_train_0 = int(len(idx_0) * split_ratio)
n_train_1 = int(len(idx_1) * split_ratio)

# Indices train y test
train_idx = np.concatenate((idx_0[:n_train_0], idx_1[:n_train_1]))
test_idx = np.concatenate((idx_0[n_train_0:], idx_1[n_train_1:]))

# Shuffle final para mezclar clases
np.random.shuffle(train_idx)
np.random.shuffle(test_idx)

X_train_raw = X_raw[train_idx]
y_train = y[train_idx]
X_test_raw = X_raw[test_idx]
y_test = y[test_idx]

print(f"Train size: {len(X_train_raw)} ({np.mean(y_train):.2f} postive rate)")
print(f"Test size: {len(X_test_raw)} ({np.mean(y_test):.2f} positive rate)")

# 3. Normalización Manual (Fit en train, Transform en train y test)
mean_train = np.mean(X_train_raw, axis=0)
std_train = np.std(X_train_raw, axis=0)

# Evitar división por cero
std_train[std_train == 0] = 1.0

X_train_norm = (X_train_raw - mean_train) / std_train
X_test_norm = (X_test_raw - mean_train) / std_train

# Agregar columna de unos (bias term / intercepto)
X_train = np.hstack([np.ones((X_train_norm.shape[0], 1)), X_train_norm])
X_test = np.hstack([np.ones((X_test_norm.shape[0], 1)), X_test_norm])

print("Normalización completada. Se agregó término de bias (columna de 1s).")
print(f"Forma final X_train: {X_train.shape}")
print(f"Forma final X_test: {X_test.shape}")


### Explicación del Preprocesamiento
- **Normalización**: Es crítica para Gradient Descent. Al normalizar, la superficie de costo se vuelve más esférica (en lugar de elipses alargadas), permitiendo que el algoritmo converja más rápido y con un paso de aprendizaje ($\alpha$) más estable.
- **Split Estratificado**: Garantiza que tanto el conjunto de entrenamiento como el de prueba sean representativos de la población real. Si hiciéramos un split aleatorio simple en un dataset pequeño y desbalanceado, podríamos acabar con un test set que casi no tenga casos positivos, falseando la evaluación.


## 4. Implementación de Logistic Regression desde Cero

Implementaremos las funciones nucleares del algoritmo.

### Modelo Matemático
La hipótesis de la regresión logística está dada por la función sigmoide aplicada a una combinación lineal de los inputs:
$$ h_w(x) = \sigma(w^T x) = \frac{1}{1 + e^{-(w_0 + w_1 x_1 + ... + w_n x_n)}} $$

La función de costo (Log Loss) para $m$ ejemplos es:
$$ J(w) = - \frac{1}{m} \sum_{i=1}^{m} [ y^{(i)} \log(h_w(x^{(i)})) + (1 - y^{(i)}) \log(1 - h_w(x^{(i)})) ] $$

El gradiente de la función de costo respecto a los pesos es:
$$ \frac{\partial J(w)}{\partial w_j} = \frac{1}{m} \sum_{i=1}^{m} (h_w(x^{(i)}) - y^{(i)}) x_j^{(i)} $$


In [ ]:
def sigmoid(z):
    '''
    Calcula la función sigmoide 1 / (1 + e^-z).
    Mayor estabilidad numérica utilizando np.exp(-z) o condicionales si z es muy negativo.
    '''
    # Clip para evitar overflow/underflow
    z = np.clip(z, -500, 500)
    return 1.0 / (1.0 + np.exp(-z))

def compute_cost(y, y_hat):
    '''
    Calcula el costo Log Loss (Cross-Entropy).
    y: etiquetas reales (binarias)
    y_hat: probabilidades predichas (0 a 1)
    '''
    m = len(y)
    # epsilon pequeño para evitar log(0)
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    
    cost = - (1/m) * np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    return cost

def compute_gradients(X, y, y_hat):
    '''
    Calcula el gradiente dJ/dw.
    X: Matriz de features (m, n+1)
    y: vector etiquetas reales
    y_hat: vector predicciones
    '''
    m = len(y)
    # Error: predicción - real
    error = y_hat - y
    # Gradiente: (1/m) * X.T dot error
    gradients = (1/m) * np.dot(X.T, error)
    return gradients

def gradient_descent(X, y, alpha, iterations):
    '''
    Ejecuta el bucle de optimización.
    X: Features
    y: Labels
    alpha: Learning rate
    iterations: Número de iteraciones
    '''
    m, n = X.shape
    # Inicializar pesos en ceros
    w = np.zeros(n)
    cost_history = []
    
    print(f"Iniciando entrenamiento con alpha={alpha}, iters={iterations}...")
    
    for i in range(iterations):
        # 1. Forward pass (Cálculo de z y h(x))
        z = np.dot(X, w)
        y_hat = sigmoid(z)
        
        # 2. Calcular costo
        cost = compute_cost(y, y_hat)
        cost_history.append(cost)
        
        # 3. Calcular gradientes
        grads = compute_gradients(X, y, y_hat)
        
        # 4. Actualizar pesos (w = w - alpha * dw)
        w = w - alpha * grads
        
        # Reporte cada cierto tiempo
        if i % (iterations // 10) == 0:
            print(f"Iteración {i}: Costo = {cost:.4f}")
            
    return w, cost_history


In [ ]:
# Configuración de hiperparámetros
ALPHA = 0.1
ITERATIONS = 2000

# Entrenamiento
w_final, cost_history = gradient_descent(X_train, y_train, ALPHA, ITERATIONS)

print(f"\nPesos finales entrenados: {w_final}")

# Visualización de la convergencia
plt.figure(figsize=(8, 5))
plt.plot(range(ITERATIONS), cost_history, color='blue')
plt.title('Convergencia del Costo (J) vs Iteraciones')
plt.xlabel('Iteraciones')
plt.ylabel('Costo Log Loss')
plt.grid(True)
plt.show()


### Análisis del Entrenamiento
- **Curva de Costo**: Observamos si la curva desciende suavemente y se aplana (converge). Si el costo oscila o sube, el learning rate ($\alpha$) podría ser muy alto. Si baja muy lento, podría ser muy bajo.
- **Interpretación de Pesos $w$**:
    - El peso $w_0$ es el sesgo (intercepto).
    - Los pesos $w_1, ..., w_n$ corresponden a cada feature normalizada. La magnitud indica la fuerza de la influencia y el signo indica la dirección (positivo: aumenta riesgo, negativo: disminuye riesgo) relativo a la media.


## 5. Predicción y Evaluación

Implementaremos la función `predict` usando un umbral de decisión de 0.5. Evaluaremos el modelo usando métricas estándar.


In [ ]:
# Implementación manual de métricas para evitar dependencia de sklearn
def accuracy_score(y_true, y_pred):
    return np.mean(y_true == y_pred)

def confusion_matrix(y_true, y_pred):
    # [[TN, FP], [FN, TP]]
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    return np.array([[TN, FP], [FN, TP]])

def precision_score(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    return TP / (TP + FP) if (TP + FP) > 0 else 0.0

def recall_score(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    return TP / (TP + FN) if (TP + FN) > 0 else 0.0

def f1_score(y_true, y_pred):
    p = precision_score(y_true, y_pred)
    r = recall_score(y_true, y_pred)
    return 2 * (p * r) / (p + r) if (p + r) > 0 else 0.0


def predict(X, w, threshold=0.5):
    '''
    Predice clases 0 o 1 dado X y pesos w.
    '''
    z = np.dot(X, w)
    probs = sigmoid(z)
    predictions = (probs >= threshold).astype(int)
    return predictions

# Predicciones en Train y Test
y_train_pred = predict(X_train, w_final)
y_test_pred = predict(X_test, w_final)

# Cálculo de métricas
metrics_data = {
    'Dataset': ['Train', 'Test'],
    'Accuracy': [accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred)],
    'Precision': [precision_score(y_train, y_train_pred), precision_score(y_test, y_test_pred)],
    'Recall': [recall_score(y_train, y_train_pred), recall_score(y_test, y_test_pred)],
    'F1 Score': [f1_score(y_train, y_train_pred), f1_score(y_test, y_test_pred)]
}

metrics_df = pd.DataFrame(metrics_data)
display(metrics_df)

print("\nMatriz de Confusión (Test):")
print(confusion_matrix(y_test, y_test_pred))


### Interpretación Clínica
- **High Recall (Sensibilidad)**: Es deseable en medicina para no dejar pasar casos enfermos (falsos negativos bajos).
- **High Precision**: Importante para no alarmar pacientes sanos (falsos positivos bajos).
- **Overfitting/Underfitting**: Comparamos métricas de Train vs Test. Si Train >> Test, hay overfitting.


## 6. Visualización de Fronteras de Decisión

Visualizaremos cómo el modelo separa las clases en un plano 2D. Como tenemos múltiples dimensiones, tomaremos pares de features, entrenaremos un modelo **solo con esos dos features** y graficaremos la línea de separación.


In [ ]:
def plot_decision_boundary_2d(X_2d, y, feature_names):
    '''
    Entrena un modelo simple en 2D y grafica la frontera.
    '''
    # Normalizar features locales
    mean_2d = np.mean(X_2d, axis=0)
    std_2d = np.std(X_2d, axis=0)
    X_norm = (X_2d - mean_2d) / std_2d
    
    # Agregar bias
    X_bias = np.hstack([np.ones((X_norm.shape[0], 1)), X_norm])
    
    # Entrenar modelo local
    w_2d, _ = gradient_descent(X_bias, y, alpha=0.1, iterations=1000)
    
    # Crear Grid para plot
    x_min, x_max = X_norm[:, 0].min() - 0.5, X_norm[:, 0].max() + 0.5
    y_min, y_max = X_norm[:, 1].min() - 0.5, X_norm[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05),
                         np.arange(y_min, y_max, 0.05))
    
    # Predecir sobre todo el grid
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    grid_bias = np.hstack([np.ones((grid_points.shape[0], 1)), grid_points])
    Z = predict(grid_bias, w_2d)
    Z = Z.reshape(xx.shape)
    
    # Plot
    plt.figure(figsize=(7, 6))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    plt.scatter(X_norm[:, 0], X_norm[:, 1], c=y, edgecolors='k', cmap='coolwarm')
    plt.title(f'Frontera de decisión: {feature_names[0]} vs {feature_names[1]}')
    plt.xlabel(feature_names[0] + ' (normalizada)')
    plt.ylabel(feature_names[1] + ' (normalizada)')
    plt.show()

# Pares a visualizar
pairs = [
    ('Age', 'Max HR'),
    ('Cholesterol', 'BP'),
    ('Age', 'Cholesterol')
]

# Indices originales de estas features en X_raw/selected_features
feat_indices = {name: i for i, name in enumerate(selected_features)}

for f1, f2 in pairs:
    idx1, idx2 = feat_indices[f1], feat_indices[f2]
    X_subset = X_raw[:, [idx1, idx2]] # Usar todos los datos para ilustrar
    print(f"\nEntrenando y graficando para: {f1} vs {f2}...")
    plot_decision_boundary_2d(X_subset, y, [f1, f2])


### Análisis de Fronteras
Observamos que una frontera lineal (recta) puede no ser suficiente para separar perfectamente las clases complejas. Sin embargo, nos da una idea clara de la separabilidad lineal de las variables seleccionadas.


## 7. Regularización L2 (Ridge)

Implementaremos regularización L2 para penalizar pesos grandes y prevenir overfitting.

Nueva función de costo:
$$ J(w) = J_{original}(w) + \frac{\lambda}{2m} \sum_{j=1}^{n} w_j^2 $$
(Nota: usualmente no se regulariza el bias $w_0$).

Nuevo gradiente:
$$ J(w) = J_{original}(w) + \frac{\lambda}{2m} \sum_{j=1}^{n} w_j^2 $$


In [ ]:
def compute_cost_l2(y, y_hat, w, lambda_reg, m):
    # Costo base
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    base_cost = - (1/m) * np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    
    # Término L2 (excluyendo bias w[0])
    l2_term = (lambda_reg / (2 * m)) * np.sum(np.square(w[1:]))
    
    return base_cost + l2_term

def gradient_descent_l2(X, y, alpha, iterations, lambda_reg):
    m, n = X.shape
    w = np.zeros(n)
    cost_history = []
    
    for i in range(iterations):
        z = np.dot(X, w)
        y_hat = sigmoid(z)
        
        cost = compute_cost_l2(y, y_hat, w, lambda_reg, m)
        cost_history.append(cost)
        
        error = y_hat - y
        gradients = (1/m) * np.dot(X.T, error)
        
        # Agregar regularización al gradiente (excepto para w[0])
        # w[0] no se penaliza
        gradients[1:] += (lambda_reg / m) * w[1:]
        
        w = w - alpha * gradients
        
    return w, cost_history

# Comparación de lambdas
lambdas = [0, 0.1, 1, 10, 50]
colors = ['red', 'blue', 'green', 'orange', 'purple']

plt.figure(figsize=(10, 6))

for i, lam in enumerate(lambdas):
    w_l2, costs_l2 = gradient_descent_l2(X_train, y_train, ALPHA, ITERATIONS, lam)
    label_name = f'Lambda={lam} (Norma w: {np.linalg.norm(w_l2[1:]):.2f})'
    plt.plot(costs_l2, label=label_name, color=colors[i])

plt.title('Comparación de Costo con Regularización L2')
plt.xlabel('Iteraciones')
plt.ylabel('Costo (Regularizado)')
plt.legend()
plt.ylim(0.3, 0.7) # Zoom para ver diferencias
plt.show()    


### Impacto de L2
Observamos que a medida que aumenta $\lambda$ (Lambda), la magnitud de los pesos (Norma w) disminuye. Esto reduce la varianza del modelo (reduce overfitting) a costa de agregar algo de sesgo (underfitting si $\lambda$ es muy grande).


## 8. Conclusiones Finales

### Aprendizajes Clave
1.  **Implementación desde Cero**: Comprender la derivada del gradiente y cómo se actualizan los pesos desmitifica las "cajas negras" de librerías como scikit-learn.
2.  **Importancia del Preprocesamiento**: Sin normalización, el Gradient Descent convergería muy lentamente o divergiría, especialmente con features de escalas dispares como `Cholesterol` vs `ST depression`.
3.  **Regularización**: Vimos cómo penalizar la magnitud de los pesos ayuda a mantener un modelo más simple y generalizable.

### Efectividad del Modelo
El modelo de regresión logística lineal logró un rendimiento razonable (ver métricas arriba). Dado que es un problema médico, la interpretabilidad de los pesos es una gran ventaja sobre modelos más complejos como Redes Neuronales.

### Limitaciones y Trabajo Futuro
- **Linealidad**: La regresión logística asume una frontera de decisión lineal. Si las clases no son linealmente separables, el modelo sufre. Se podría mejorar agregando **features polinómicas** (interacciones entre variables).
- **Outliers**: El modelo es sensible a outliers. Un análisis más profundo de limpieza de datos podría mejorar métricas.


# 9. Preparación para SageMaker

Para desplegar este modelo en Amazon SageMaker, necesitamos:
1. Guardar los pesos entrenados (`w_final`).
2. Crear un script de inferencia (`inference.py`) que le diga a SageMaker cómo cargar los pesos y hacer predicciones.
3. Empaquetar todo en un archivo `model.tar.gz`.


In [ ]:
import os
import tarfile
import numpy as np

# 1. Guardar los pesos del modelo
np.save('model_weights.npy', w_final)
print("Pesos guardados en 'model_weights.npy'")

# 2. Crear script de inferencia (inference.py)
inference_code = """
import numpy as np
import json
import os

def model_fn(model_dir):
    # Carga el modelo desde el directorio de artefactos
    print("Cargando modelo desde: " + model_dir)
    weights = np.load(os.path.join(model_dir, "model_weights.npy"))
    return weights

def input_fn(request_body, request_content_type):
    # Procesa la entrada. Espera JSON.
    if request_content_type == 'application/json':
        input_data = json.loads(request_body)
        return input_data
    else:
        raise ValueError("Este modelo solo soporta application/json")

def predict_fn(input_data, model):
    # Realiza la predicción
    # model es el vector de pesos w_final
    data = np.array(input_data)
    
    # Manejo de dimensiones
    if data.ndim == 1:
        data = data.reshape(1, -1)
        
    # Agregar Sesgo (Bias) - Columna de 1s al inicio
    # Asumimos que el cliente envía clean features sin bias
    m = data.shape[0]
    data_bias = np.hstack([np.ones((m, 1)), data])
    
    # Función Sigmoide
    z = np.dot(data_bias, model)
    # Clip para estabilidad
    z = np.clip(z, -500, 500)
    prob = 1.0 / (1.0 + np.exp(-z))
    
    return prob

def output_fn(prediction, response_content_type):
    # Formatea la salida
    return json.dumps(prediction.tolist())
"""

with open('inference.py', 'w') as f:
    f.write(inference_code)
print("Script 'inference.py' creado.")

# 3. Crear model.tar.gz
with tarfile.open('model.tar.gz', "w:gz") as tar:
    tar.add('model_weights.npy', arcname='model_weights.npy')
    tar.add('inference.py', arcname='inference.py')

print("Archivo 'model.tar.gz' creado exitosamente. Este archivo se sube a S3.")


## 10. Despliegue Automático con SageMaker SDK

Si prefieres no usar la consola manual, puedes ejecutar el despliegue directamente desde aquí usando el SDK de Python de SageMaker (`sagemaker`).

**Requisitos Previos**:
1.  Tener una cuenta de AWS y credenciales configuradas en tu máquina (archivo `~/.aws/credentials` o variables de entorno).
2.  Tener un **IAM Role** en AWS con permisos de `AmazonSageMakerFullAccess` y acceso a S3.
3.  Instalar librerías de AWS: `pip install sagemaker boto3`.


In [ ]:
import sagemaker
import boto3
from sagemaker import Model

# --- CONFIGURACIÓN ---
# REEMPLAZA ESTO con el ARN de tu rol de IAM. Ejemplo: 'arn:aws:iam::123456789012:role/service-role/AmazonSageMaker-ExecutionRole-2020...'
role_arn = "arn:aws:iam::TU_CUENTA_ID:role/TU_ROL_SAGEMAKER" 

if role_arn == "arn:aws:iam::TU_CUENTA_ID:role/TU_ROL_SAGEMAKER":
    print("⚠️ DEBES ACTUALIZAR LA VARIABLE 'role_arn' CON TU ROL DE AWS ANTES DE CONTINUAR")
else:
    # Sesión y Bucket
    sess = sagemaker.Session()
    bucket = sess.default_bucket()
    region = sess.boto_region_name
    prefix = 'heart-disease-lr'
    
    # 1. Subir model.tar.gz a S3
    model_artifact = sess.upload_data(path='model.tar.gz', bucket=bucket, key_prefix=prefix)
    print(f"Artefacto subido a: {model_artifact}")
    
    # 2. Definir imagen del contenedor (Usamos sklearn como base ligera)
    # Recuperamos la URI de la imagen pre-construida de AWS
    image_uri = sagemaker.image_uris.retrieve(
        framework="sklearn",
        region=region,
        version="1.0-1",
        image_scope="inference"
    )
    
    # 3. Crear Objeto Modelo
    model = Model(
        image_uri=image_uri,
        model_data=model_artifact,
        role=role_arn,
        sagemaker_session=sess,
        name="heart-disease-custom-lr"
    )
    
    # 4. Desplegar Endpoint
    print("Desplegando endpoint... (esto toma 5-10 mins)")
    predictor = model.deploy(
        initial_instance_count=1,
        instance_type='ml.t2.medium', # Instancia económica
        endpoint_name='heart-disease-endpoint-manual'
    )
    
    print(f"\nEndpoint desplegado: {predictor.endpoint_name}")


## 11. Simulación de Despliegue Local (Prueba de Concepto)

Antes de subir a la nube, es vital probar que nuestros scripts (`inference.py`) y artefactos (`model.tar.gz`) funcionan correctamente. Esta sección simula el entorno de SageMaker ejecutando todo **localmente en tu máquina**.

**¿Qué hace este código?**
1. Descomprime el archivo `model.tar.gz` (simulando que el servidor lo descargó de S3).
2. Importa dinámicamente tu script `inference.py`.
3. Ejecuta la función `model_fn` para cargar el modelo.
4. Ejecuta `predict_fn` con datos de prueba para ver si funciona.


In [ ]:
import tarfile
import importlib.util
import sys
import os

# 1. Simular entorno: Crear carpeta temporal para descomprimir
local_deploy_dir = 'local_deployment'
if not os.path.exists(local_deploy_dir):
    os.makedirs(local_deploy_dir)

print(f"Descomprimiendo model.tar.gz en {local_deploy_dir}...")
with tarfile.open('model.tar.gz', 'r:gz') as tar:
    tar.extractall(path=local_deploy_dir)

# 2. Cargar dinámicamente el script inference.py
inference_script_path = os.path.join(local_deploy_dir, 'inference.py')
spec = importlib.util.spec_from_file_location("inference", inference_script_path)
inference = importlib.util.module_from_spec(spec)
sys.modules["inference"] = inference
spec.loader.exec_module(inference)
print("Script inference.py cargado exitosamente como módulo.")

# 3. Ejecutar el flujo de inferencia
print("\n--- Iniciando Prueba de Inferencia ---")

# A. Cargar Modelo
model = inference.model_fn(local_deploy_dir)
print(f"Modelo cargado. Pesos: {model[:5]}... (truncado)")

# B. Datos de prueba (Un paciente ficticio)
# Normalizamos igual que en el training (usando media/std de ejemplo o 0 para simplificar)
# Ejemplo: [Age, Sex, BP, Cholesterol, Max HR, ST depression] (valores normalizados)
test_input = [0.5, 1.0, 0.2, -0.5, -1.0, 2.0]
print(f"Input (paciente): {test_input}")

# C. Predecir
prediction = inference.predict_fn(test_input, model)
print(f"Probabilidad predicha: {prediction.item():.4f}")

# D. Formato de Salida
output = inference.output_fn(prediction, 'application/json')
print(f"Respuesta JSON final: {output}")

print("\n✅ ¡El despliegue local funciona correctamente! El modelo está listo para AWS.")
